# Sentiment analysis for SST-2 with DiltilBERT

## Install HuggingFace dependencies

In [ ]:
!pip install -q datasets
!pip install -q transformers

     |████████████████████████████████| 325 kB 16.0 MB/s 
     |████████████████████████████████| 1.1 MB 65.2 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 212 kB 64.0 MB/s 
     |████████████████████████████████| 136 kB 74.5 MB/s 
     |████████████████████████████████| 127 kB 76.8 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 76.4 MB/s 
     |████████████████████████████████| 271 kB 74.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 3.8 MB 8.3 MB/s 
     |████████████████████████████████| 895 kB 61.3 MB/s 
     |████████████████████████████████| 596 kB 41.6 MB/s 
     |████████████████████████████████| 6.

## Create functions to download and preproccess data

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import CanineTokenizer, DataCollatorWithPadding, DistilBertTokenizer

bz = 16  # batch size

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def encode(samples):
    return tokenizer(samples['sentence'], padding=False, truncation=True)

def quantify(samples):
    return {'label': np.digitize(samples['label'], np.array([.5]), right=True)}

letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
def typo(samples, typo_percent=0):
    for i,sample in enumerate(samples['sentence']):
        for j,x in enumerate(sample):
            if np.random.random() < typo_percent and x.isalpha():
                aux = list(samples['sentence'][i])
                aux[j] = np.random.choice(letters)
                samples['sentence'][i] = ''.join(aux)
    return samples

def generate_dataset(typo_percent=0):
    #dataset = load_dataset('sst')
    dataset = load_dataset('glue', 'sst2')

    # Set validation as test
    dataset['test'] = dataset['validation']
    # Split train in train and valid
    dataset_aux = dataset['train'].train_test_split(0.05, seed=0)
    dataset['train'] = dataset_aux['train']
    dataset['validation'] = dataset_aux['test']

    if typo_percent > 0:
        dataset = dataset.map(typo, batched=True, batch_size=bz, fn_kwargs={'typo_percent':typo_percent})
    dataset = dataset.map(encode, batched=True, batch_size=bz)
    
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    loader = {}
    for split in dataset:
        loader[split] = DataLoader(dataset[split], batch_size=bz, shuffle=(split=='train'), collate_fn=DataCollatorWithPadding(tokenizer))
    
    return dataset, loader

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Create train and eval functions

In [ ]:
from tqdm import tqdm
import copy

def train_canine(net, device, optimizer, criterion, lr, lr_scheduler, train_loader, val_loader, epochs, unfreeze=False, unfreeze_ep=5):
    best_loss = np.Inf
    best_ep = 1
    iters = []
    train_losses = []
    val_losses = []
    log_interval = len(train_loader)//5

    for ep in range(epochs):
        with tqdm(total=len(train_loader), unit_scale=True, postfix={'Loss':0.0, 'Accuracy':0.0, 'lr':lr_scheduler.get_last_lr()[0]},
                        desc="Epoch : %i/%i" % (ep+1, epochs)) as pbar:
            net.train()
            total_loss = 0.0
            correct = 0
            nb_samples = 0
            for it, batch in enumerate(train_loader):
                # Converting to cuda tensors
                batch = {k: v.to(device) for k, v in batch.items()}
                seq, attn_masks, labels = \
                    batch['input_ids'], batch['attention_mask'], batch['labels']
                labels = labels.to(torch.int64)
        
                # Clear gradients
                optimizer.zero_grad()

                # Forward pass
                output = net(input_ids=seq, attention_mask=attn_masks, labels=labels)

                # Backpropagating the gradients
                loss = output.loss #HUGGINGFACE
                #loss = criterion(output, labels)
                loss.backward()
                # Optimization step
                optimizer.step()

                total_loss += loss.item()

                logits = output.logits #HUGGINGFACE
                #logits = output
                predicted = logits.argmax(axis=1)
                correct += predicted.eq(labels).sum().item()
                nb_samples += len(predicted)

                pbar.set_postfix({'Loss':total_loss/(it+1), 'Accuracy':100*correct/nb_samples, 'lr':lr_scheduler.get_last_lr()[0]})
                pbar.update(1)
                
                lr_scheduler.step()

                if it % log_interval == log_interval-1:
                    val_loss, val_acc = evaluate_loss(net, device, val_loader, criterion)  # Compute validation loss
                    print(f"Val Loss : {val_loss:.3f}. Val Accuracy {val_acc:.2f}%")

                    if val_loss < best_loss:
                        print(f"Best validation loss improved from {best_loss:.3f} to {val_loss:.3f}")
                        print()
                        net_copy = copy.deepcopy(net)  # save a copy of the model
                        best_loss = val_loss
                        best_ep = ep + 1

        val_loss, val_acc = evaluate_loss(net, device, val_loader, criterion)  # Compute validation loss
        print()
        print(f"Epoch {ep+1} complete! Val Loss : {val_loss:.3f}. Val Accuracy {val_acc:.2f}%")

        train_losses.append(total_loss/len(train_loader))
        val_losses.append(val_loss)

        if val_loss < best_loss:
            print(f"Best validation loss improved from {best_loss:.3f} to {val_loss:.3f}")
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1
        
        if unfreeze and ep == unfreeze_ep:
            for param in net.parameters():
                param.requires_grad = True

    # Saving the model
    path_to_model=f'models/CANINE_lr_{lr}_val_loss_{round(best_loss, 5)}_ep_{best_ep}_typo_{typo_percent}.pt'
    torch.save(net_copy.state_dict(), path_to_model)
    print(f"The model has been saved in {path_to_model}")

    del loss
    torch.cuda.empty_cache()

    return train_losses, val_losses, net_copy

In [ ]:
def evaluate_loss(net, device, dataloader, criterion):
    net.eval()

    total_loss = 0
    correct = 0

    print('Evaluating...')
    with torch.no_grad():
        for it, batch in enumerate(dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            seq, attn_masks, labels = \
                batch['input_ids'], batch['attention_mask'], batch['labels']
            labels = labels.to(torch.int64)
            output = net(seq, attn_masks, labels=labels)
            total_loss += output.loss.item() #HUGGINGFACE
            #total_loss += criterion(output, labels).item()
            predicted = output.logits.argmax(axis=1) #HUGGINGFACE
            #predicted = output.argmax(axis=1)
            correct += predicted.eq(labels).sum().item()

    return total_loss/len(dataloader), 100*correct/len(dataloader.dataset)

## Train the model with different levels of noise

In [ ]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...


In [ ]:
#freeze_canine = False  # if True, freeze the encoder weights and only update the classification layer weights
#maxlen = 2048  # maximum length of the tokenized input pair : if greater than "maxlen", the input is truncated and else if smaller, the input is padded
lr = 2e-5  # learning rate
epochs = 5  # number of training epochs

In [ ]:
from transformers import CanineTokenizer, CanineConfig, CanineForSequenceClassification, get_linear_schedule_with_warmup, DistilBertForSequenceClassification#, AdamW
from torch.optim import AdamW

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
losses = []
accs = []

for typo_percent in [0, 0.05, 0.1, 0.2, 0.4]:
    dataset, loader = generate_dataset(typo_percent=typo_percent)

    net = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

    net.to(device)

    optimizer = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss()
    t_total = len(loader['train']) * epochs  # Necessary to take into account Gradient accumulation
    num_warmup_steps = int(t_total*0.1) # The number of steps for the warmup phase.
    lr_scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

    train_losses, val_losses, net = train_canine(net, device, optimizer, criterion, lr, lr_scheduler, loader['train'], loader['validation'], epochs)

    print(f'\n### Typo percentage = {typo_percent} ###')
    losses_aux = []
    accs_aux = []
    for split in ['train', 'validation', 'test']:
        loss, acc = evaluate_loss(net, device, loader[split], criterion)
        losses_aux.append(loss)
        accs_aux.append(acc)
        print()
        print(split)
        print(f'   loss = {loss}\n   acc = {acc}')

    losses.append(losses_aux)
    accs.append(accs_aux)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Evaluating...


Epoch : 5/5: 100%|█████████▉| 4.00k/4.00k [06:16<00:01, 1.03it/s, Loss=0.056, Accuracy=98.1, lr=3.33e-9]

Val Loss : 1.587. Val Accuracy 65.47%


Epoch : 5/5: 100%|██████████| 4.00k/4.00k [06:16<00:00, 10.6it/s, Loss=0.056, Accuracy=98.1, lr=1.11e-9]


Evaluating...

Epoch 5 complete! Val Loss : 1.587. Val Accuracy 65.47%
The model has been saved in models/CANINE_lr_2e-05_val_loss_0.61115_ep_2_typo_0.4.pt

### Typo percentage = 0.4 ###
Evaluating...

train
   loss = 0.44711435142607353
   acc = 81.08188368421875
Evaluating...

validation
   loss = 0.6111504476499784
   acc = 66.41923990498812
Evaluating...

test
   loss = 0.6025622877207669
   acc = 67.88990825688073


## Print final results

In [ ]:
from tabulate import tabulate

noises = [0, 0.05, 0.1, 0.2, 0.4]
for i in range(len(noises)):
    accs[i] = [noises[i]]+accs[i]
    losses[i] = [noises[i]]+losses[i]

print('Accuracy table')
print(tabulate([['Noise', 'Train', 'Validation', 'Test']]+accs))

print('\n\nLosses table')
print(tabulate([['Noise', 'Train', 'Validation', 'Test']]+losses))